#Deep Learning con Python
`Autor: Erwing FC 
~erwingforerocastro@gmail.com`

In [1]:
import keras
import numpy as np
path = keras.utils.get_file('nietzsche.txt',
                            origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('Tamaño del cuerpo:', len(text))

606208/600901 [==============================] - 0s 1us/step
Tamaño del cuerpo: 600893


###Preparacion de los datos

In [2]:
maxlen = 60 #secuencias de 60 caracteres
step = 3    #nueva secuencia cada 3 caracteres
sentences = [] #secuencias
next_chars = [] #palabra siguiente de esa secuencia

for i in range(0, len(text) - maxlen, step): #(0,600833,3)
  sentences.append(text[i: i + maxlen])      # por ejemplo: text[0:60] tomara caracteres de 0 hasta 59
  next_chars.append(text[i + maxlen])        # por ejemplo: text[60] tomara el caracter 60
print('Numero de secuencias:', len(sentences))

chars = sorted(list(set(text))) #lista de unicos caracteres en el cuerpo
print('Caracteres unicos:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars) #lista de caracteres con su indice 

print('Vectorización...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) 
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

#se realiza one-hot encoding
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

Numero de secuencias: 200278
Caracteres unicos: 57
Vectorización...


###Estructura y compilación

In [3]:
from keras import layers

net = keras.models.Sequential()
net.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
net.add(layers.Dense(len(chars), activation='softmax'))

#compilacion
optimizer = keras.optimizers.RMSprop(lr=0.01)
net.compile(loss='categorical_crossentropy', optimizer=optimizer)

###Predicciones

Funcion de temperatura:
<br><br>
<img src="https://i.stack.imgur.com/HYyQT.jpg">


In [4]:
#Función para muestrear el siguiente caracter dadas las predicciones del modelo

def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature #logNat(x)/1
  exp_preds = np.exp(preds)           #e^x
  preds = exp_preds / np.sum(exp_preds) 
  probas = np.random.multinomial(1, preds, 1) #probabilidad
  return np.argmax(probas) #la maxima probabilidad

In [ ]:
import random
import sys

for epoch in range(1, 60): #se entrena el modelo por 60 epocas
    print('epoca', epoch)
    net.fit(x, y, batch_size=128, epochs=1) #se entrena el modelo con solo 1 epoca
    #seleccionamos una seccion de texto aletoria
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generando con semilla: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]: #se intenta una gama de diferentes temperaturas de muestreo
        print('------ temperatura:', temperature)
        sys.stdout.write(generated_text)
        for i in range(400): #Genera 400 caracteres, comenzando por el texto semilla

            sampled = np.zeros((1, maxlen, len(chars)))  #one-hot encoding del texto
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            #realizamos una prediccion
            preds = net.predict(sampled, verbose=0)[0]

            #probabilidad del siguiente caracter
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            #agregamos el caracter predicho
            generated_text += next_char
            generated_text = generated_text[1:] #cambiamos el texto quitamos la primera palabra y dejamos la siguiente
            sys.stdout.write(next_char)